## Mutational screening of functional proteins


#### **Mutational screen of all esssential proteins**

In [ ]:
import pandas as pd
import os
import pickle
data_path = './data/essential_proteins.csv'
ess_prot_AFdb = (
    pd.read_csv(data_path)['AlphaFoldDB']
    .dropna()  
    .str.replace(';', '')  
    .apply(lambda x: f'AF-{x}-F1.pdb') 
)
ess_prot_AFdb = ess_prot_AFdb.tolist()

path_pdb = './data/pdbs'
path_pqr=  './data/pqrs'
output_path = './data/test'

Mutator_dict = {}
count = 1
no_pdb = []
for entry in ess_prot_AFdb:
    if os.path.exists(os.path.join(path_pdb, entry)):
        Mutator_dict[entry] = prot_mut(path_pdb, entry, path_pqr, Deep_mut=False, iterations=100, cutoff_value = -0.005, threshhold = 1000, seed = 0)
        print(f'{entry} finished ({count}/{len(ess_prot_AFdb)})')
        count += 1
    else:
        no_pdb.append(entry)
        print(f'{os.path.join(path_pdb,entry)} not found')


with(open('./data/Mutator_rational.pkl', 'wb')) as file:
    pickle.dump(Mutator_dict, file)


#### **Analyze mutation sreen of essential proteins**

In [ ]:
import pickle
with(open('./data/Essential_genes_mutated.pkl', 'rb')) as f:
    Mutator_dict_load = pickle.load(f)
with(open('./data/Mutator_dict_13first_bs_error.pkl', 'rb')) as f:
    Mutator_dict_13 = pickle.load(f)
Mutated_proteins = {**Mutator_dict_load, **Mutator_dict_13}

""" with(open('./data/Mutator_rational_test_top79.pkl', 'rb')) as file:
    Mutated_proteins = pickle.load(file) """


Increase = 0
No_improvement = []
Mutated_improvement = {}
for key in Mutated_proteins:
    if Mutated_proteins[key][0][0][0] == Mutated_proteins[key][0][1][0]:
        No_improvement.append(key)
    else:
        Mutated_improvement[key] = Mutated_proteins[key]

print(Mutated_improvement.keys())

Diff = 0
Changed_aa_amount = 0
highest_increase = ('name',0)
biggest_relative_change = ('name',0)
biggest_absolute_chagne = ('name',0)
rel_mut_increase = ('name',0)
for key in Mutated_improvement:
    #Melt point increase
    Diff += float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0])
    
    #highest increase
    if float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]) > highest_increase[1]:
        highest_increase = (key, float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]))
    
    #percentage of changed aminoacids
    changed_aa = 0
    for n in range(len(Mutated_improvement[key][1][0])): 
        if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
            changed_aa += 1
    if changed_aa/len(Mutated_improvement[key][1][0]) > biggest_relative_change[1]:
        biggest_relative_change = (key, changed_aa/len(Mutated_improvement[key][1][0]))
    if changed_aa > biggest_absolute_chagne[1]:
        biggest_absolute_chagne = (key, changed_aa)
    Changed_aa_amount += changed_aa/len(Mutated_improvement[key][1][0])
    if changed_aa > 0:
        if (float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]))/float(changed_aa) > rel_mut_increase[1]:
            rel_mut_increase = (key, (float(Mutated_improvement[key][0][1][0][0]) - float(Mutated_improvement[key][0][0][0][0]))/float(changed_aa))

#Mutation list for edge cases
highest_increase_mut_list = []
biggest_rel_change_mut_list = []
biggest_abs_change_mut_list =[]
highest_mut2melt_list = []
for key in Mutated_improvement:
    if key == highest_increase[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                highest_increase_mut_list.append(f'{n+1}')
    if key == biggest_relative_change[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                biggest_rel_change_mut_list.append(f'{n+1}')
    if key == biggest_absolute_chagne[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                biggest_abs_change_mut_list.append(f'{n+1}')
    if key == rel_mut_increase[0]:
        for n in range(len(Mutated_improvement[key][1][0])):
            if Mutated_improvement[key][1][0][n] != Mutated_improvement[key][1][1][n]:
                highest_mut2melt_list.append(f'{n+1}')


#Print results
#print(f'Proteins without a pdb file: {len(no_pdb)}')
print(f"Proteins that couldn't be improved: {len(No_improvement)}")
print(f'Proteins that could be improved: {len(Mutated_improvement.keys())} \n')
print(f'Average meltPoint increase: {Diff/len(Mutated_improvement.keys())}')
print(f'Average percentage of changed aminoacid per protein: {Changed_aa_amount/len(Mutated_improvement.keys())} \n')
print(f'Highest meltPoint increase: {highest_increase[0]} with {highest_increase[1]} ({Mutated_improvement[highest_increase[0]][0][0][0][0]} -> {Mutated_improvement[highest_increase[0]][0][1][0][0]})\nWT sequence:      {''.join(Mutated_improvement['AF-P37948-F1.pdb'][1][0])} \nMutated sequence: {''.join(Mutated_improvement['AF-P37948-F1.pdb'][1][1])}')
print(f'{'+'.join(highest_increase_mut_list)}\n')
print(f'Highest percentage of changed aminoacid: {biggest_relative_change[0]} with {biggest_relative_change[1]} with {int(len(Mutated_improvement[biggest_relative_change[0]][1][0])*biggest_relative_change[1])} of {len(Mutated_improvement[biggest_relative_change[0]][1][0])}\nWT sequence:      {''.join(Mutated_improvement['AF-Q47710-F1.pdb'][1][0])} \nMutated sequence: {''.join(Mutated_improvement['AF-Q47710-F1.pdb'][1][1])}')
print(f'{'+'.join(biggest_rel_change_mut_list)}\n')
print(f'Highest absolute number of changed aminoacid: {biggest_absolute_chagne[0]} with {biggest_absolute_chagne[1]} of {len(Mutated_improvement[biggest_absolute_chagne[0]][1][0])}\nWT sequence:      {''.join(Mutated_improvement['AF-P46918-F1.pdb'][1][0])} \nMutated sequence: {''.join(Mutated_improvement['AF-P46918-F1.pdb'][1][1])}')
print(f'{'+'.join(biggest_abs_change_mut_list)}\n')                                                                                                                                                                                                                                         
print(f'')
print(f'Biggest meltPoint change per mutation {rel_mut_increase[0]} with {rel_mut_increase[1]} ({Mutated_improvement[rel_mut_increase[0]][0][0][0][0]} -> {Mutated_improvement[rel_mut_increase[0]][0][1][0][0]})\nWT sequence:      {''.join(Mutated_improvement[rel_mut_increase[0]][1][0])} \nMutated sequence: {''.join(Mutated_improvement[rel_mut_increase[0]][1][1])}')
print(f'{'+'.join(highest_mut2melt_list)}\n')


#### **Create dataframe from mutation results for statistical testing and plotting**

In [ ]:
import pickle
import pandas as pd
with(open('./data/Essential_genes_mutated.pkl', 'rb')) as f:
    Mutator_dict_load = pickle.load(f)
with(open('./data/Mutator_dict_13first_bs_error.pkl', 'rb')) as f:
    Mutator_dict_13 = pickle.load(f)
Mutated_proteins = {**Mutator_dict_load, **Mutator_dict_13}
Mutated_df = pd.DataFrame(columns = ['Protein', 'WT_prediction', 'Mut_prediction', 'WT_sequence', 'Mut_sequence', 'Mutations'])
for key in Mutated_proteins:
    wt_prediction = Mutated_proteins[key][0][0][0][0]
    mut_prediction = Mutated_proteins[key][0][1][0][0]
    wt_seq = ''.join(Mutated_proteins[key][1][0])
    mut_seq = ''.join(Mutated_proteins[key][1][1])
    mutations_list = []
    for i in range(len(wt_seq)):
        if wt_seq[i] != mut_seq[i]:
            mutations_list.append(f'{wt_seq[i]}{i+1}{mut_seq[i]}')
    new_row = [key.split('-')[1], wt_prediction, mut_prediction, wt_seq, mut_seq, mutations_list]
    #print(new_row)
    Mutated_df.loc[len(Mutated_df)] = new_row

print(Mutated_df.loc[1])

with(open('./data/Mutated_proteins_df.pkl', 'wb')) as f:
    pickle.dump(Mutated_df, f)